In [13]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import pandas as pd
import joblib
from rapidfuzz import process

In [14]:
# Load model and label encoder
model = joblib.load("disease_prediction_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

In [15]:
# Load training data
train_data = pd.read_csv("Training.csv")
if "Unnamed: 133" in train_data.columns:
    train_data.drop("Unnamed: 133", axis=1, inplace=True)
symptoms_list = list(train_data.columns[:-1])

In [16]:
# Load medication data
medications_df = pd.read_csv("medications.csv")

In [17]:
root = tk.Tk()
root.title("Disease Prediction System")
root.geometry("800x600")

''

In [ ]:
try:
    bg_img = Image.open(r"img.jpg")  
    bg_img = bg_img.resize((800, 600))
    bg_photo = ImageTk.PhotoImage(bg_img)
    bg_label = tk.Label(root, image=bg_photo)
    bg_label.bg_photo = bg_photo  
    bg_label.place(relwidth=1, relheight=1)
except Exception as e:
    print("Background image could not be loaded:", e)

In [19]:
predicted_disease = tk.StringVar()
recommended_medicine = tk.StringVar()

In [20]:
def clean_symptom(user_symptom):
    best_match, score, _ = process.extractOne(user_symptom, symptoms_list)
    return best_match if score >= 70 else None

In [21]:
# Predict disease
def predict_disease():
    user_input = symptom_entry.get().lower().replace(",", " ").split()
    cleaned_symptoms = []
    for sym in user_input:
        match = clean_symptom(sym)
        if match:
            cleaned_symptoms.append(match)

    if not cleaned_symptoms:
        messagebox.showerror("Error", "No valid symptoms detected!")
        return

    input_data = [0] * len(symptoms_list)
    for symptom in cleaned_symptoms:
        input_data[symptoms_list.index(symptom)] = 1

    prediction = model.predict([input_data])[0]
    disease_name = label_encoder.inverse_transform([prediction])[0]

    predicted_disease.set(f"Predicted Disease: {disease_name}")
    recommended_medicine.set("") 

In [22]:
# Recommend medicine
def recommend_medicine():
    if not predicted_disease.get():
        messagebox.showwarning("Warning", "Please predict a disease first!")
        return

    disease_name = predicted_disease.get().replace("Predicted Disease: ", "")
    medicine_row = medications_df[medications_df["Disease"].str.lower() == disease_name.lower()]

    if not medicine_row.empty:
        medicine = medicine_row["Medication"].values[0]
        recommended_medicine.set(f"Recommended Medicine: {medicine}")
    else:
        recommended_medicine.set("No medicine found for this disease.")

In [23]:
# ------------------ UI Components ------------------ #
# Title
title_label = tk.Label(root, text="Disease Prediction System",font=("Arial", 20, "bold"), bg="#ffffff", fg="#2c3e50")
title_label.place(relx=0.5, y=30, anchor="center")

# Symptom label
symptom_label = tk.Label(root, text="Enter Symptoms (comma or space separated):", font=("Arial", 12), bg="#ffffff")
symptom_label.place(relx=0.5, y=100, anchor="center")

# Symptom entry
symptom_entry = tk.Entry(root, font=("Arial", 12), width=50)
symptom_entry.place(relx=0.5, y=130, anchor="center")

# Predict button
predict_button = tk.Button(root, text="🔍 Predict Disease", font=("Arial", 12, "bold"),command=predict_disease, bg="#2196f3", fg="white", width=20)
predict_button.place(relx=0.35, y=180, anchor="center")

# Recommend Medicine button
medicine_button = tk.Button(root, text="💊 Recommend Medicine", font=("Arial", 12, "bold"),command=recommend_medicine, bg="#ff9800", fg="white", width=20)
medicine_button.place(relx=0.65, y=180, anchor="center")

# Result: Disease
result_label = tk.Label(root, textvariable=predicted_disease, font=("Arial", 14, "bold"),bg="#ffffff", fg="black")
result_label.place(relx=0.5, y=250, anchor="center")

# Result: Medicine
medicine_label = tk.Label(root, textvariable=recommended_medicine, font=("Arial", 14, "bold"),bg="#ffffff", fg="green")
medicine_label.place(relx=0.5, y=300, anchor="center")

# ------------------ Run Tkinter Loop ------------------ #
root.mainloop()
